## *Multisig accounts
#### 03.7 Writing Smart Contracts
##### Peter Gruber (peter.gruber@usi.ch)
2024-02-14 (started 2022-02-02)

- Create and use k-out-of-n multisignature accounts

## Setup
Starting with chapter 3.4, the lines below will always automatically load ...
* The functions in `algo_util.py`
* The accounts MyAlgo, Alice and Bob
* The Purestake credentials

In [1]:
# Loading shared code and credentials
import sys, os
codepath = '..'+os.path.sep+'..'+os.path.sep+'sharedCode'
sys.path.append(codepath)
from algo_util import *
cred = load_credentials()

# Shortcuts to directly access the main accounts
MyAlgo  = cred['MyAlgo']
Alice   = cred['Alice']
Bob     = cred['Bob']
Charlie = cred['Charlie']
Dina    = cred['Dina']

In [2]:
from algosdk import account, mnemonic
from algosdk.v2client import algod
from algosdk.transaction import PaymentTxn, MultisigTransaction
import algosdk.error
import json

In [3]:
print(MyAlgo['public'])
print(Alice['public'])
print(Bob['public'])
print(Charlie['public'])
print(Dina['public'])

WSC24MVUSQ32IZYD7FNN54Z44IXWL4X7BOJD6AGFOCHOG4PDFESLZUGLTI
ALICEXOA4Q2OD5CKBYND4UX75K3TAODEC3XCVNQ3URMKUMZKUOTOSAQLIU
BOB23JBQLW3AJREEG3KD7ULMAGYGVZ2LVF2OMZ2XBJIR64NO5P24XN7WEU
CHARLS5INIT5KOLDK3F2QAZXY7N4W7GOE6UTXHEHS3RBNCR2WJXRRTH3GY
DINAAZWBJM6DRY73CWK7IOA4S7C6PVXQMQ4DJ3TK3DAGAGFJTX5KEYESDI


### Connecting to the Algorand testnet

In [4]:
algod_client = algod.AlgodClient(algod_token='', algod_address=cred['algod_test'], headers=cred['api_token'])
algod_client.status()["last-round"]

38992208

## Multisig account
Alice, Bob and Charlie create a multisig account, 
with the requirement that at least 2 out of 3 have to sign.

#### Step 1: Create the Multisig account
* Using the new command `Multisig`
* Create a 2-out-of-3 multisig
    * Set `threshold` to 2
    * Add 3 accounts to the `account_list`

In [5]:
from algosdk.transaction import Multisig

version   = 1     # multisig version, currently always 1
threshold = 2     # min 2 signatures required
account_list = [Alice['public'], Bob['public'], Charlie['public']]

msig = Multisig(version, threshold, account_list)
msig_address = msig.address()

print("Multisig Address: ", msig_address)

Multisig Address:  PVRODHMQD2SQIV4NXEFYCZYTV365WNCQ6HVCRLWQ3QTK3NAJUKUCDWMJKY


#### Step 2: Fund the multisig account
* Can be funded via PaymentTxn
* Here we fund from a dispenser

In [6]:
print('https://dispenser.testnet.aws.algodev.network/?account='+msig_address)

https://dispenser.testnet.aws.algodev.network/?account=PVRODHMQD2SQIV4NXEFYCZYTV365WNCQ6HVCRLWQ3QTK3NAJUKUCDWMJKY


#### Step 3.1: Create a simple payment transaction
* Together, they want to send 1.2 ALGOs to Dina

In [7]:
sp = algod_client.suggested_params()
amt = int(1.2*1E6)

txn = PaymentTxn(sender=msig_address,         # <-- From Multisig address
                 sp=sp, 
                 receiver=Dina['public'], 
                 amt = amt
                )

#### Step 3.2: Convert simple transaction to a multisig transaction and sign it
* Alice and Bob sign the transaction

In [8]:
# Create multisig transaction
mtx = MultisigTransaction(txn, msig)

# Collect at least 2 signatures
mtx.sign(Alice['private'])
mtx.sign(Bob['private'])

#### Step 3.3: Send the transaction

In [9]:
txid = algod_client.send_transaction(mtx)

#### Step 3.4: Wait for confirmation

In [10]:
txinfo = wait_for_confirmation(algod_client, txid)

Current round is  38992260.
Waiting for round 38992260 to finish.
Waiting for round 38992261 to finish.
Transaction 5SCZG6XB5FWTH23KXP2ECF5ZSYAOXS32HXXHPIXB2EEQIX3DIG4A confirmed in round 38992262.


#### Check Dina's holdings

In [12]:
algod_client.account_info(Dina["public"])["amount"] / 1e6

20.324351

#### Exercise
* Repeat steps 3.1-3.4 with Alice and Charlie signing the transaction
* Repeat steps 3.1-3.4 with Alice and Dina signing 
* Repeat steps 3.1-3.4 with only Alice signing

#### Exercise
* Create a 1-out-of-4 multisig, for which Alice, Bob, Charlie and/or Dina can sign
* Transfer 1 ALGO to `MyAlgo`